In [ ]:
# RDF Solution environment
import MDAnalysis as mda
from MDAnalysis.analysis import rdf
import MDAnalysis.transformations as trans
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['font.size'] = 14
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.weight']='bold'
%matplotlib inline

# Load the universe (topology and trajectory)
u = mda.Universe('../SOLUTION_MD/P-2O/04_NPT/npt_1.tpr', '../SOLUTION_MD/P-2O/04_NPT/solution.gro')
transforms = [trans.unwrap(u.atoms)]
u.trajectory.add_transformations(*transforms)
water = u.select_atoms("resname H2O")
NMP = u.select_atoms("resname COS")
AA = u.select_atoms("resname DON")

# Define a class to set atom groups to coarse-grain (CG) representations
class CG_group:
    def __init__(self, ag):
        self.ag = ag
        self.universe = self.ag.universe
        self.trajectory = self.ag.universe.trajectory
    @property
    def positions(self):
        return self.ag.center_of_mass(unwrap=True, compound="residues")
    def __len__(self):
        return self.ag.center_of_mass(unwrap=True, compound="residues").shape[0]
    
# decide functional groups
def get_target_atoms(label):
    if label == "S-Main":
        return CG_group(u.select_atoms("resname PT* and name S2"))
    elif label == "S-Side":
        return u.select_atoms("resname PT* and name S1 S3")
    elif label == "2O":
        return u.select_atoms("resname P2O and name O3 O4")
    elif label == "DBZ-SO2":
        sulfone1 = u.select_atoms("resname PT* and name S5 O3 O4")
        sulfone3 = u.select_atoms("resname P2O and name S1 O1 O2")
        return CG_group(sulfone1+sulfone3)
    elif label == "TSO-SO2":
        sulfone2 = u.select_atoms("resname PT* and name S4 O1 O2")
        return CG_group(sulfone2)
    else:
        raise ValueError(f"Unknown target_atoms label: {label}")

# plot results
colors = ['#2e4057', '#048ba8', '#f18f01']
def plot_rdf(water_rdf, AA_rdf, NMP_rdf, name):
    plt.plot(AA_rdf.results.bins, AA_rdf.results.rdf, label = 'AA', color=colors[1])
    plt.plot(NMP_rdf.results.bins, NMP_rdf.results.rdf, label='NMP', color=colors[2])
    plt.plot(water_rdf.results.bins, water_rdf.results.rdf, label = 'Water', color=colors[0])
    plt.xlabel('Radius (Å)', fontsize = 20)
    plt.ylabel('g(r)', fontsize = 20)
    plt.yticks(fontsize = 16)
    plt.xticks(ticks=range(0, 21, 5), fontsize=16)
    plt.title(f'{name}', fontsize = 22)
    plt.legend(fontsize=12, loc = 'upper left',frameon=False)
    plt.tight_layout()
    plt.show()

target_atoms_configs = ['S-Main','S-Side','2O','DBZ-SO2','TSO-SO2']
names = ['S-Main','S-Side','2O', r"DBS-SO$_2$",r"BDTT-SO$_2$"]
water_cg = CG_group(water)
AA_cg = CG_group(AA)
NMP_cg = CG_group(NMP)

# Analyze the radial distribution function (RDF) of solution molecules with respect to a functional group
for i,label in enumerate(target_atoms_configs):
    print(f"Analyzing {label}...")
    cg = get_target_atoms(label)
    water_rdf = rdf.InterRDF(cg, water_cg, exclusion_block=(1,1), range=(0.0, 20.0))
    AA_rdf = rdf.InterRDF(cg, AA_cg, exclusion_block=(1,1), range=(0.0, 20.0))
    NMP_rdf = rdf.InterRDF(cg, NMP_cg, exclusion_block=(1,1), range=(0.0, 20.0))
    water_rdf.run(verbose=True)
    AA_rdf.run(verbose=True)
    NMP_rdf.run(verbose=True)
    print(f"Finished {label}!")
    plot_rdf(water_rdf, AA_rdf, NMP_rdf, names[i])

